In [4]:
import pandas as pd
import glob, os
from biopandas.mol2 import PandasMol2
from biopandas.mol2 import split_multimol2
import numpy as np
import os
import numpy as np
from scipy.spatial import distance

pdmol = PandasMol2()

def save_e3_mols (file, e3_glob):
    with open(f'{file}_filtered.mol2', 'w') as f:
        for emol2 in e3_glob:
            e3_pd=PandasMol2().read_mol2_from_list(mol2_lines=emol2[1], mol2_code=emol2[0])
            f.write(e3_pd.mol2_text)
        
        
for pp in glob.glob('/groups/cherkasvgrp/share/progressive_docking/hmslati/protacs/manual_linker_disjoint_propose_docking/*_pp_docking'):
    print(pp)
    e3_glob=glob.glob(f'{pp}/e3_*p1fixed.mol2')[0]
    e3=glob.glob(f'{pp}/e3_*p1fixed.mol2')[0].split('/')[-1].split('.')[0]
    poi_glob=glob.glob(f'{pp}/poi_*p1fixed.mol2')[0]
    poi=glob.glob(f'{pp}/poi_*p1fixed.mol2')[0].split('/')[-1].split('.')[0]
    df_scores=pd.read_csv(f'{pp}/pp_scores.csv', sep=',')
    try:
        os.remove(f'{pp}/{poi}_filtered.mol2')
    except:
        pass
    try: 
        os.remove(f'{pp}/{e3}_filtered.mol2')
    except:
        pass
    try:
        os.remove(f'{pp}/pp_scores_filtered.csv')
    except:
        pass
    with open(f'{pp}/{poi}_filtered.mol2', 'w') as f:
        for pmol2 in split_multimol2(f'{poi_glob}'):
            poi_pd=PandasMol2().read_mol2_from_list(mol2_lines=pmol2[1], mol2_code=pmol2[0])
            f.write(poi_pd.mol2_text)
            pligand_coor=poi_pd.df[poi_pd.df['subst_name'].str.contains("UNL")].iloc[:,2:5].values
            with open(f'{pp}/{e3}_filtered.mol2', 'w') as f:
                print(poi)
                list_e3=[]
                #keys=['cityblock']
                #distanc_lis={key: [] for key in keys}
               #print(distanc_lis)
                distanc_lis=[]
                
                for emol2 in split_multimol2(f'{e3_glob}'):
                        keep_molecule=False
                        e3_pd=PandasMol2().read_mol2_from_list(mol2_lines=emol2[1], mol2_code=emol2[0])

                        # do some analysis

                        keto_coord = e3_pd.df[e3_pd.df['atom_name'] == 'O95'][['x', 'y', 'z']]
                        if keto_coord.empty:
                            keto_coord = e3_pd.df[e3_pd.df['atom_name'] == 'N95'][['x', 'y', 'z']]
                        if keto_coord.empty:
                            keto_coord = e3_pd.df[e3_pd.df['atom_name'] == 'C95'][['x', 'y', 'z']]
                        if keto_coord.empty:
                            keto_coord = e3_pd.df[e3_pd.df['atom_name'] == 'S95'][['x', 'y', 'z']]
                        if keto_coord.empty:
                            keto_coord = e3_pd.df[e3_pd.df['atom_name'] == 'P95'][['x', 'y', 'z']]

                        distances = poi_pd.distance(keto_coord.values[0])
                        poi_pd.df['distances'] = distances

                        dis99=poi_pd.df[poi_pd.df['atom_name'].str.contains("O94|N94|C94|S94|P94")].distances.values

                        
                        eligand_coor=e3_pd.df[e3_pd.df['subst_name'].str.contains("UNL")].iloc[:,2:5].values
                        #possibilities=["braycurtis", "canberra", "chebyshev", 'cityblock', "cosine", "euclidean", "mahalanobis", "minkowski", "seuclidean", 'sqeuclidean']
                        distanc_lis.append((dis99+np.mean(distance.cdist(pligand_coor, eligand_coor, "cityblock"))/2))
                        #distanc_lis.append(dis99)
                            ##distance_centroid=distance.cdist(pligand_coor, eligand_coor, 'euclidean')
                        #print(pligand_coor.shape, eligand_coor.shape,distance_centroid.shape)
                        #distance_centroid=np.linalg.norm(pligand_coor - eligand_coor)
                        #print(distance_centroid)
                        ##distanc_lis.append(min(min(distance_centroid, key=min)))
                        #print(min(min(distance_centroid, key=min)))


#                         if 0<float(dis.item())<float(i):
#                             #print(dis.item())
#                             keep_molecule=True
#                             list_e3.append(e3)
#                             #print(e3)








                        # save molecule if it passes our filter criterion
                        #if keep_molecule: 
                            # note that the mol2_text contains the original mol2 content

                            #print(' do nothing')

                            #f.write(e3_pd.mol2_text) 
#             if not list_e3:
#                 print(e3, " does not have a distance")
#                 print(min(distanc_lis))
            #print(distanc_lis)
            try:
                a=np.asarray(distanc_lis).reshape(1,100)
            except:
                a=np.asarray(distanc_lis)[:,1:].reshape(1,100)
            ind=np.argpartition(a[0], 19)[:20].tolist()
            print (ind)
            multimol2=[list(split_multimol2(f'{e3_glob}'))[i] for i in ind]
            file=f'{pp}/{e3}'
            save_e3_mols(file,multimol2)
            pp_scores=df_scores.iloc[ind]
            pp_scores.to_csv(f'{pp}/pp_scores_filtered.csv', index=False)
            print(pp)
          
            break
            
            


/groups/cherkasvgrp/share/progressive_docking/hmslati/protacs/manual_linker_disjoint_propose_docking/poi_5ehr_13_docked_rec_e3_vhl_5ehr_13_docked_rec_pp_docking
poi_5ehr_13_docked_rec_r_prep_pp_p1fixed
[74, 73, 40, 55, 15, 5, 63, 13, 28, 2, 27, 49, 66, 33, 90, 91, 84, 39, 82, 60]
/groups/cherkasvgrp/share/progressive_docking/hmslati/protacs/manual_linker_disjoint_propose_docking/poi_5ehr_13_docked_rec_e3_vhl_5ehr_13_docked_rec_pp_docking
/groups/cherkasvgrp/share/progressive_docking/hmslati/protacs/manual_linker_disjoint_propose_docking/poi_4qnq_39_docked_rec_e3_cereblon_4qnq_39_docked_rec_pp_docking
poi_4qnq_39_docked_rec_r_prep_pp_p1fixed
[21, 28, 97, 51, 1, 46, 73, 74, 31, 24, 10, 58, 23, 84, 69, 90, 32, 17, 59, 27]
/groups/cherkasvgrp/share/progressive_docking/hmslati/protacs/manual_linker_disjoint_propose_docking/poi_4qnq_39_docked_rec_e3_cereblon_4qnq_39_docked_rec_pp_docking
/groups/cherkasvgrp/share/progressive_docking/hmslati/protacs/manual_linker_disjoint_propose_docking/poi_

[7, 21, 56, 76, 22, 82, 47, 83, 2, 26, 67, 80, 40, 33, 12, 62, 93, 97, 88, 66]
/groups/cherkasvgrp/share/progressive_docking/hmslati/protacs/manual_linker_disjoint_propose_docking/poi_5t8e_4_docked_rec_e3_vhl_5t8e_4_docked_rec_pp_docking
/groups/cherkasvgrp/share/progressive_docking/hmslati/protacs/manual_linker_disjoint_propose_docking/poi_5eu1_3_docked_rec_e3_vhl_5eu1_3_docked_rec_pp_docking
poi_5eu1_3_docked_rec_r_prep_pp_p1fixed
[91, 75, 59, 29, 63, 89, 94, 7, 72, 78, 62, 11, 16, 37, 36, 27, 3, 23, 92, 52]
/groups/cherkasvgrp/share/progressive_docking/hmslati/protacs/manual_linker_disjoint_propose_docking/poi_5eu1_3_docked_rec_e3_vhl_5eu1_3_docked_rec_pp_docking
/groups/cherkasvgrp/share/progressive_docking/hmslati/protacs/manual_linker_disjoint_propose_docking/poi_4ci3_29_docked_rec_e3_vhl_4ci3_29_docked_rec_pp_docking
poi_4ci3_29_docked_rec_r_prep_pp_p1fixed
[61, 37, 98, 27, 95, 5, 93, 90, 8, 23, 80, 78, 56, 76, 58, 15, 62, 36, 63, 26]
/groups/cherkasvgrp/share/progressive_dockin

[18, 86, 69, 61, 71, 57, 74, 1, 45, 43, 41, 36, 35, 85, 27, 15, 88, 92, 65, 60]
/groups/cherkasvgrp/share/progressive_docking/hmslati/protacs/manual_linker_disjoint_propose_docking/poi_3lq5_18_docked_rec_e3_cereblon_3lq5_18_docked_rec_pp_docking
/groups/cherkasvgrp/share/progressive_docking/hmslati/protacs/manual_linker_disjoint_propose_docking/poi_6nuq_9_docked_rec_e3_cereblon_6nuq_9_docked_rec_pp_docking
poi_6nuq_9_docked_rec_r_prep_pp_p1fixed
[10, 59, 39, 38, 73, 70, 96, 53, 4, 8, 32, 83, 41, 71, 42, 9, 99, 34, 36, 76]
/groups/cherkasvgrp/share/progressive_docking/hmslati/protacs/manual_linker_disjoint_propose_docking/poi_6nuq_9_docked_rec_e3_cereblon_6nuq_9_docked_rec_pp_docking
/groups/cherkasvgrp/share/progressive_docking/hmslati/protacs/manual_linker_disjoint_propose_docking/poi_2euf_20_docked_rec_e3_vhl_2euf_20_docked_rec_pp_docking
poi_2euf_20_docked_rec_r_prep_pp_p1fixed
[49, 28, 98, 46, 89, 88, 85, 48, 82, 80, 23, 75, 22, 17, 33, 39, 60, 68, 7, 57]
/groups/cherkasvgrp/share/

poi_5gmp_16_docked_rec_r_prep_pp_p1fixed
[12, 42, 53, 96, 74, 79, 84, 43, 33, 56, 18, 29, 48, 22, 76, 65, 93, 11, 85, 66]
/groups/cherkasvgrp/share/progressive_docking/hmslati/protacs/manual_linker_disjoint_propose_docking/poi_5gmp_16_docked_rec_e3_vhl_5gmp_16_docked_rec_pp_docking
/groups/cherkasvgrp/share/progressive_docking/hmslati/protacs/manual_linker_disjoint_propose_docking/poi_4ci3_31_docked_rec_e3_vhl_4ci3_31_docked_rec_pp_docking
poi_4ci3_31_docked_rec_r_prep_pp_p1fixed
[83, 57, 54, 63, 48, 96, 68, 95, 8, 69, 37, 36, 70, 59, 79, 94, 98, 47, 2, 44]
/groups/cherkasvgrp/share/progressive_docking/hmslati/protacs/manual_linker_disjoint_propose_docking/poi_4ci3_31_docked_rec_e3_vhl_4ci3_31_docked_rec_pp_docking
/groups/cherkasvgrp/share/progressive_docking/hmslati/protacs/manual_linker_disjoint_propose_docking/poi_5l2i_3_docked_rec_e3_cereblon_5l2i_3_docked_rec_pp_docking
poi_5l2i_3_docked_rec_r_prep_pp_p1fixed
[72, 16, 35, 94, 1, 66, 69, 91, 90, 86, 70, 83, 71, 46, 30, 20, 59, 4, 

poi_5ehr_23_docked_rec_r_prep_pp_p1fixed
[14, 51, 46, 52, 42, 59, 62, 33, 72, 20, 82, 16, 12, 75, 90, 38, 99, 47, 26, 41]
/groups/cherkasvgrp/share/progressive_docking/hmslati/protacs/manual_linker_disjoint_propose_docking/poi_5ehr_23_docked_rec_e3_vhl_5ehr_23_docked_rec_pp_docking
/groups/cherkasvgrp/share/progressive_docking/hmslati/protacs/manual_linker_disjoint_propose_docking/poi_3ue4_9_docked_rec_e3_cereblon_3ue4_9_docked_rec_pp_docking
poi_3ue4_9_docked_rec_r_prep_pp_p1fixed
[17, 43, 36, 1, 4, 27, 91, 59, 23, 60, 65, 81, 45, 53, 30, 79, 21, 24, 58, 78]
/groups/cherkasvgrp/share/progressive_docking/hmslati/protacs/manual_linker_disjoint_propose_docking/poi_3ue4_9_docked_rec_e3_cereblon_3ue4_9_docked_rec_pp_docking
/groups/cherkasvgrp/share/progressive_docking/hmslati/protacs/manual_linker_disjoint_propose_docking/poi_5lli_11_docked_rec_e3_cereblon_5lli_11_docked_rec_pp_docking
poi_5lli_11_docked_rec_r_prep_pp_p1fixed
[82, 70, 58, 97, 96, 37, 94, 32, 91, 61, 23, 87, 71, 19, 44, 26

...

poi_5mo4_1_docked_rec_r_prep_pp_p1fixed
[59, 31, 2, 3, 24, 85, 98, 87, 44, 42, 52, 54, 12, 56, 94, 90, 74, 47, 35, 25]
/groups/cherkasvgrp/share/progressive_docking/hmslati/protacs/manual_linker_disjoint_propose_docking/poi_5mo4_1_docked_rec_e3_cereblon_5mo4_1_docked_rec_pp_docking
/groups/cherkasvgrp/share/progressive_docking/hmslati/protacs/manual_linker_disjoint_propose_docking/poi_6rna_1_docked_rec_e3_iapbir2_6rna_1_docked_rec_pp_docking
poi_6rna_1_docked_rec_r_prep_pp_p1fixed
[91, 72, 41, 94, 68, 77, 89, 38, 26, 5, 81, 93, 27, 16, 2, 82, 11, 92, 71, 56]
/groups/cherkasvgrp/share/progressive_docking/hmslati/protacs/manual_linker_disjoint_propose_docking/poi_6rna_1_docked_rec_e3_iapbir2_6rna_1_docked_rec_pp_docking
/groups/cherkasvgrp/share/progressive_docking/hmslati/protacs/manual_linker_disjoint_propose_docking/poi_5eu1_10_docked_rec_e3_vhl_5eu1_10_docked_rec_pp_docking
poi_5eu1_10_docked_rec_r_prep_pp_p1fixed
[0, 22, 17, 87, 1, 64, 41, 89, 28, 54, 10, 81, 59, 9, 66, 16, 3, 77, 8

poi_5g0i_7_docked_rec_r_prep_pp_p1fixed
[19, 99, 95, 87, 79, 68, 66, 58, 46, 9, 43, 42, 39, 33, 32, 28, 0, 86, 97, 17]
/groups/cherkasvgrp/share/progressive_docking/hmslati/protacs/manual_linker_disjoint_propose_docking/poi_5g0i_7_docked_rec_e3_vhl_5g0i_7_docked_rec_pp_docking
/groups/cherkasvgrp/share/progressive_docking/hmslati/protacs/manual_linker_disjoint_propose_docking/poi_6y7k_4_docked_rec_e3_cereblon_6y7k_4_docked_rec_pp_docking
poi_6y7k_4_docked_rec_r_prep_pp_p1fixed
[54, 26, 72, 3, 37, 5, 43, 95, 50, 63, 10, 91, 79, 61, 76, 88, 52, 71, 66, 48]
/groups/cherkasvgrp/share/progressive_docking/hmslati/protacs/manual_linker_disjoint_propose_docking/poi_6y7k_4_docked_rec_e3_cereblon_6y7k_4_docked_rec_pp_docking
/groups/cherkasvgrp/share/progressive_docking/hmslati/protacs/manual_linker_disjoint_propose_docking/poi_1fkh_5_docked_rec_e3_cereblon_1fkh_5_docked_rec_pp_docking
poi_1fkh_5_docked_rec_r_prep_pp_p1fixed
[64, 10, 93, 22, 98, 5, 6, 34, 70, 48, 53, 11, 40, 46, 32, 21, 67, 55, 

poi_5v5y_4_docked_rec_r_prep_pp_p1fixed
[51, 62, 2, 3, 30, 66, 39, 65, 96, 18, 44, 59, 1, 13, 87, 89, 72, 73, 71, 98]
/groups/cherkasvgrp/share/progressive_docking/hmslati/protacs/manual_linker_disjoint_propose_docking/poi_5v5y_4_docked_rec_e3_cereblon_5v5y_4_docked_rec_pp_docking
/groups/cherkasvgrp/share/progressive_docking/hmslati/protacs/manual_linker_disjoint_propose_docking/poi_3lq5_15_docked_rec_e3_cereblon_3lq5_15_docked_rec_pp_docking
poi_3lq5_15_docked_rec_r_prep_pp_p1fixed
[22, 59, 56, 76, 70, 31, 68, 50, 10, 73, 40, 39, 91, 85, 99, 42, 43, 86, 1, 57]
/groups/cherkasvgrp/share/progressive_docking/hmslati/protacs/manual_linker_disjoint_propose_docking/poi_3lq5_15_docked_rec_e3_cereblon_3lq5_15_docked_rec_pp_docking
/groups/cherkasvgrp/share/progressive_docking/hmslati/protacs/manual_linker_disjoint_propose_docking/poi_3k6p_19_docked_rec_e3_cereblon_3k6p_19_docked_rec_pp_docking
poi_3k6p_19_docked_rec_r_prep_pp_p1fixed
[80, 88, 43, 41, 72, 86, 35, 7, 34, 33, 53, 95, 63, 18, 14

poi_4qnq_67_docked_rec_r_prep_pp_p1fixed
[42, 64, 23, 18, 75, 60, 39, 13, 30, 22, 16, 85, 87, 43, 98, 17, 73, 76, 11, 24]
/groups/cherkasvgrp/share/progressive_docking/hmslati/protacs/manual_linker_disjoint_propose_docking/poi_4qnq_67_docked_rec_e3_iapbir2_4qnq_67_docked_rec_pp_docking
/groups/cherkasvgrp/share/progressive_docking/hmslati/protacs/manual_linker_disjoint_propose_docking/poi_5uvx_4_docked_rec_e3_cereblon_5uvx_4_docked_rec_pp_docking
poi_5uvx_4_docked_rec_r_prep_pp_p1fixed
[59, 82, 77, 70, 69, 2, 78, 7, 68, 47, 62, 56, 84, 85, 88, 74, 34, 12, 51, 89]
/groups/cherkasvgrp/share/progressive_docking/hmslati/protacs/manual_linker_disjoint_propose_docking/poi_5uvx_4_docked_rec_e3_cereblon_5uvx_4_docked_rec_pp_docking
/groups/cherkasvgrp/share/progressive_docking/hmslati/protacs/manual_linker_disjoint_propose_docking/poi_5g0j_4_docked_rec_e3_vhl_5g0j_4_docked_rec_pp_docking
poi_5g0j_4_docked_rec_r_prep_pp_p1fixed
[18, 84, 58, 52, 97, 48, 47, 46, 70, 76, 80, 31, 24, 13, 91, 61, 67

poi_4ci3_19_docked_rec_r_prep_pp_p1fixed
[10, 77, 80, 58, 87, 1, 46, 30, 22, 14, 78, 42, 28, 24, 72, 64, 37, 92, 11, 8]
/groups/cherkasvgrp/share/progressive_docking/hmslati/protacs/manual_linker_disjoint_propose_docking/poi_4ci3_19_docked_rec_e3_vhl_4ci3_19_docked_rec_pp_docking
/groups/cherkasvgrp/share/progressive_docking/hmslati/protacs/manual_linker_disjoint_propose_docking/poi_4im0_22_docked_rec_e3_vhl_4im0_22_docked_rec_pp_docking
poi_4im0_22_docked_rec_r_prep_pp_p1fixed
[88, 65, 73, 42, 43, 48, 91, 33, 52, 97, 39, 2, 74, 24, 7, 96, 16, 38, 8, 14]
/groups/cherkasvgrp/share/progressive_docking/hmslati/protacs/manual_linker_disjoint_propose_docking/poi_4im0_22_docked_rec_e3_vhl_4im0_22_docked_rec_pp_docking


In [10]:
#crystals
import pandas as pd
import glob, os
from biopandas.mol2 import PandasMol2
from biopandas.mol2 import split_multimol2
import numpy as np
import os
import numpy as np
from scipy.spatial import distance

pdmol = PandasMol2()

def save_e3_mols (file, e3_glob):
    with open(f'{file}_filtered.mol2', 'w') as f:
        for emol2 in e3_glob:
            e3_pd=PandasMol2().read_mol2_from_list(mol2_lines=emol2[1], mol2_code=emol2[0])
            f.write(e3_pd.mol2_text)
        
        
for pp in glob.glob('/groups/cherkasvgrp/share/progressive_docking/hmslati/protacs/manual_linker_disjoint_propose_docking/crystals/different_linker_orient/*_pp_docking'):
    print(pp)
    e3_glob=glob.glob(f'{pp}/e3_*p1fixed.mol2')[0]
    e3=glob.glob(f'{pp}/e3_*p1fixed.mol2')[0].split('/')[-1].split('.')[0]
    poi_glob=glob.glob(f'{pp}/poi_*p1fixed.mol2')[0]
    poi=glob.glob(f'{pp}/poi_*p1fixed.mol2')[0].split('/')[-1].split('.')[0]
    df_scores=pd.read_csv(f'{pp}/pp_scores.csv', sep=',')
    try:
        os.remove(f'{pp}/{poi}_filtered.mol2')
    except:
        pass
    try: 
        os.remove(f'{pp}/{e3}_filtered.mol2')
    except:
        pass
    try:
        os.remove(f'{pp}/pp_scores_filtered.csv')
    except:
        pass
    with open(f'{pp}/{poi}_filtered.mol2', 'w') as f:
        for pmol2 in split_multimol2(f'{poi_glob}'):
            poi_pd=PandasMol2().read_mol2_from_list(mol2_lines=pmol2[1], mol2_code=pmol2[0])
            f.write(poi_pd.mol2_text)
            pligand_coor=poi_pd.df[poi_pd.df['subst_name'].str.contains("UNL")].iloc[:,2:5].values
            with open(f'{pp}/{e3}_filtered.mol2', 'w') as f:
                print(poi)
                list_e3=[]
                #keys=['cityblock']
                #distanc_lis={key: [] for key in keys}
               #print(distanc_lis)
                distanc_lis=[]
                
                for emol2 in split_multimol2(f'{e3_glob}'):
                        keep_molecule=False
                        e3_pd=PandasMol2().read_mol2_from_list(mol2_lines=emol2[1], mol2_code=emol2[0])

                        # do some analysis

                        keto_coord = e3_pd.df[e3_pd.df['atom_name'] == 'O95'][['x', 'y', 'z']]
                        if keto_coord.empty:
                            keto_coord = e3_pd.df[e3_pd.df['atom_name'] == 'N95'][['x', 'y', 'z']]
                        if keto_coord.empty:
                            keto_coord = e3_pd.df[e3_pd.df['atom_name'] == 'C95'][['x', 'y', 'z']]
                        if keto_coord.empty:
                            keto_coord = e3_pd.df[e3_pd.df['atom_name'] == 'S95'][['x', 'y', 'z']]
                        if keto_coord.empty:
                            keto_coord = e3_pd.df[e3_pd.df['atom_name'] == 'P95'][['x', 'y', 'z']]

                        distances = poi_pd.distance(keto_coord.values[0])
                        poi_pd.df['distances'] = distances

                        dis99=poi_pd.df[poi_pd.df['atom_name'].str.contains("O94|N94|C94|S94|P94")].distances.values

                        
                        eligand_coor=e3_pd.df[e3_pd.df['subst_name'].str.contains("UNL")].iloc[:,2:5].values
                        #possibilities=["braycurtis", "canberra", "chebyshev", 'cityblock', "cosine", "euclidean", "mahalanobis", "minkowski", "seuclidean", 'sqeuclidean']
                        distanc_lis.append((dis99+np.mean(distance.cdist(pligand_coor, eligand_coor, "cityblock"))/2))
                        #distanc_lis.append(dis99)
                            ##distance_centroid=distance.cdist(pligand_coor, eligand_coor, 'euclidean')
                        #print(pligand_coor.shape, eligand_coor.shape,distance_centroid.shape)
                        #distance_centroid=np.linalg.norm(pligand_coor - eligand_coor)
                        #print(distance_centroid)
                        ##distanc_lis.append(min(min(distance_centroid, key=min)))
                        #print(min(min(distance_centroid, key=min)))


#                         if 0<float(dis.item())<float(i):
#                             #print(dis.item())
#                             keep_molecule=True
#                             list_e3.append(e3)
#                             #print(e3)








                        # save molecule if it passes our filter criterion
                        #if keep_molecule: 
                            # note that the mol2_text contains the original mol2 content

                            #print(' do nothing')

                            #f.write(e3_pd.mol2_text) 
#             if not list_e3:
#                 print(e3, " does not have a distance")
#                 print(min(distanc_lis))
            #print(distanc_lis)
            try:
                a=np.asarray(distanc_lis).reshape(1,100)
            except:
                a=np.asarray(distanc_lis)[:,1:].reshape(1,100)
            ind=np.argpartition(a[0], 19)[:20].tolist()
            print (ind)
            multimol2=[list(split_multimol2(f'{e3_glob}'))[i] for i in ind]
            file=f'{pp}/{e3}'
            save_e3_mols(file,multimol2)
            pp_scores=df_scores.iloc[ind]
            pp_scores.to_csv(f'{pp}/pp_scores_filtered.csv', index=False)
            print(pp)
          
            break
            
            



/groups/cherkasvgrp/share/progressive_docking/hmslati/protacs/manual_linker_disjoint_propose_docking/crystals/different_linker_orient/poi_6boy_docked_rec_e3_6boy_docked_rec_pp_docking
poi_6boy_docked_rec_r_prep_pp_p1fixed
[14, 51, 60, 24, 29, 70, 91, 57, 67, 9, 49, 32, 93, 13, 94, 35, 89, 97, 74, 19]
/groups/cherkasvgrp/share/progressive_docking/hmslati/protacs/manual_linker_disjoint_propose_docking/crystals/different_linker_orient/poi_6boy_docked_rec_e3_6boy_docked_rec_pp_docking
/groups/cherkasvgrp/share/progressive_docking/hmslati/protacs/manual_linker_disjoint_propose_docking/crystals/different_linker_orient/poi_6hr2_docked_rec_e3_6hr2_docked_rec_pp_docking
poi_6hr2_docked_rec_r_prep_pp_p1fixed
[5, 82, 24, 10, 85, 29, 70, 65, 55, 9, 3, 62, 86, 13, 49, 88, 35, 59, 32, 18]
/groups/cherkasvgrp/share/progressive_docking/hmslati/protacs/manual_linker_disjoint_propose_docking/crystals/different_linker_orient/poi_6hr2_docked_rec_e3_6hr2_docked_rec_pp_docking
/groups/cherkasvgrp/share/prog

In [87]:
## crystals

import glob, os
from biopandas.mol2 import PandasMol2
from biopandas.mol2 import split_multimol2
import numpy as np
import pandas as pd
from scipy.spatial import distance

pdmol = PandasMol2()

def save_e3_mols (file, e3_glob):
    with open(f'{file}_filtered.mol2', 'w') as f:
        for emol2 in e3_glob:
            e3_pd=PandasMol2().read_mol2_from_list(mol2_lines=emol2[1], mol2_code=emol2[0])
            f.write(e3_pd.mol2_text)
        
        
for pp in glob.glob('/groups/cherkasvgrp/share/progressive_docking/hmslati/protacs/manual_linker_disjoint_propose_docking/crystals/*_pp_docking'):
    print(pp)
    e3_glob=glob.glob(f'{pp}/e3_*p3fixed.mol2')[0]
    e3=glob.glob(f'{pp}/e3_*p3fixed.mol2')[0].split('/')[-1].split('.')[0]
    poi_glob=glob.glob(f'{pp}/poi_*p3fixed.mol2')[0]
    poi=glob.glob(f'{pp}/poi_*p3fixed.mol2')[0].split('/')[-1].split('.')[0]
    df_scores=pd.read_csv(f'{pp}/pp_scores.csv', sep=',')
    
    os.remove(f'{pp}/{poi}_filtered.mol2')
    with open(f'{pp}/{poi}_filtered.mol2', 'w') as f:
        for pmol2 in split_multimol2(f'{poi_glob}'):
            poi_pd=PandasMol2().read_mol2_from_list(mol2_lines=pmol2[1], mol2_code=pmol2[0])
            f.write(poi_pd.mol2_text)
            pligand_coor=poi_pd.df[poi_pd.df['subst_name'].str.contains("UNL")].iloc[:,2:5].values
            os.remove(f'{pp}/{e3}_filtered.mol2')
            with open(f'{pp}/{e3}_filtered.mol2', 'w') as f:
                print(poi)
                list_e3=[]
                #keys=['cityblock']
                #distanc_lis={key: [] for key in keys}
               #print(distanc_lis)
                distanc_lis=[]
                
                for emol2 in split_multimol2(f'{e3_glob}'):
                        keep_molecule=False
                        e3_pd=PandasMol2().read_mol2_from_list(mol2_lines=emol2[1], mol2_code=emol2[0])

                        # do some analysis

                        keto_coord = e3_pd.df[e3_pd.df['atom_name'] == 'O99'][['x', 'y', 'z']]
                        if keto_coord.empty:
                            keto_coord = e3_pd.df[e3_pd.df['atom_name'] == 'N99'][['x', 'y', 'z']]
                        if keto_coord.empty:
                            keto_coord = e3_pd.df[e3_pd.df['atom_name'] == 'C99'][['x', 'y', 'z']]

                        distances = poi_pd.distance(keto_coord.values[0])
                        poi_pd.df['distances'] = distances

                        dis99=poi_pd.df[poi_pd.df['atom_name'].str.contains("O99|N99|C99")].distances.values

                        
                        eligand_coor=e3_pd.df[e3_pd.df['subst_name'].str.contains("UNL")].iloc[:,2:5].values
                        #possibilities=["braycurtis", "canberra", "chebyshev", 'cityblock', "cosine", "euclidean", "mahalanobis", "minkowski", "seuclidean", 'sqeuclidean']
                        distanc_lis.append((dis99+np.mean(distance.cdist(pligand_coor, eligand_coor, "cityblock"))/2))
                        #distanc_lis.append(dis99)
                            ##distance_centroid=distance.cdist(pligand_coor, eligand_coor, 'euclidean')
                        #print(pligand_coor.shape, eligand_coor.shape,distance_centroid.shape)
                        #distance_centroid=np.linalg.norm(pligand_coor - eligand_coor)
                        #print(distance_centroid)
                        ##distanc_lis.append(min(min(distance_centroid, key=min)))
                        #print(min(min(distance_centroid, key=min)))


#                         if 0<float(dis.item())<float(i):
#                             #print(dis.item())
#                             keep_molecule=True
#                             list_e3.append(e3)
#                             #print(e3)








                        # save molecule if it passes our filter criterion
                        #if keep_molecule: 
                            # note that the mol2_text contains the original mol2 content

                            #print(' do nothing')

                            #f.write(e3_pd.mol2_text) 
#             if not list_e3:
#                 print(e3, " does not have a distance")
#                 print(min(distanc_lis))
            #print(distanc_lis)

            a=np.asarray(distanc_lis).reshape(1,100)
            ind=np.argpartition(a[0], 19)[:20].tolist()
            print (ind)
            multimol2=[list(split_multimol2(f'{e3_glob}'))[i] for i in ind]
            file=f'{pp}/{e3}'
            save_e3_mols(file,multimol2)
            pp_scores=df_scores.iloc[ind]
            pp_scores.to_csv(f'{pp}/pp_scores_filtered.csv', index=False)
            print(pp)
            
            break
            
            


/groups/cherkasvgrp/share/progressive_docking/hmslati/protacs/manual_linker_disjoint_propose_docking/crystals/poi_6boy_docked_rec_e3_6boy_docked_rec_pp_docking
poi_6boy_docked_rec_r_prep_pp_p1fixed_p2fixed_p3fixed
[30, 23, 25, 54, 68, 5, 6, 52, 17, 80, 14, 56, 40, 8, 42, 92, 89, 11, 13, 2]
/groups/cherkasvgrp/share/progressive_docking/hmslati/protacs/manual_linker_disjoint_propose_docking/crystals/poi_6boy_docked_rec_e3_6boy_docked_rec_pp_docking
/groups/cherkasvgrp/share/progressive_docking/hmslati/protacs/manual_linker_disjoint_propose_docking/crystals/poi_6hr2_docked_rec_e3_6hr2_docked_rec_pp_docking
poi_6hr2_docked_rec_r_prep_pp_p1fixed_p2fixed_p3fixed
[88, 34, 50, 63, 72, 49, 13, 7, 8, 33, 18, 73, 31, 2, 65, 74, 96, 98, 58, 59]
/groups/cherkasvgrp/share/progressive_docking/hmslati/protacs/manual_linker_disjoint_propose_docking/crystals/poi_6hr2_docked_rec_e3_6hr2_docked_rec_pp_docking
/groups/cherkasvgrp/share/progressive_docking/hmslati/protacs/manual_linker_disjoint_propose_dock

In [8]:
import prolif

ImportError: MDAnalysis not installed properly. This can happen if your C extensions have not been built.

In [5]:
import sys
sys.path.append('/home/hmslati/.conda/envs/py39/lib/python3.7/site-packages/')

In [7]:
sys.path

['/usr/lib64/python36.zip',
 '/usr/lib64/python3.6',
 '/usr/lib64/python3.6/lib-dynload',
 '',
 '/home/hmslati/.local/lib/python3.6/site-packages',
 '/usr/local/lib/python3.6/site-packages',
 '/usr/lib64/python3.6/site-packages',
 '/usr/lib/python3.6/site-packages',
 '/home/hmslati/.local/lib/python3.6/site-packages/IPython/extensions',
 '/home/hmslati/.ipython',
 '/home/hmslati/.conda/envs/py39/lib/python3.7/site-packages/']

In [7]:
pip install cutil

Defaulting to user installation because normal site-packages is not writeable
  Created wheel for cutil: filename=cutil-3.0.2-py3-none-any.whl size=16828 sha256=dde524fac31023dbda7048a98e656b1772d163aec20e769139511dfd95b3185e
  Stored in directory: /home/hmslati/.cache/pip/wheels/d5/60/c8/b3008a4211ce49cfecf5c0e431d2d2e85dca8f1e9c471bcd43
Successfully built cutil
You should consider upgrading via the '/groups/cherkasvgrp/share/progressive_docking/pd_python/tensorflow_gpu/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
